In [6]:
from pyannote.audio import Pipeline
import torch
import os

api_key = os.getenv("HUGGING_TOKEN")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=api_key
)

if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print("cuda is available")
else:
    print('cuda is not available')

/Users/yoohoyeong/Desktop/AI_Practic/venv/lib/python3.12/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


cuda is not available


In [7]:
diarization = pipeline("../audio/싼기타_비싼기타.mp3")

with open("../audio/싼기타_비싼기타.rttm", "w", encoding='utf-8') as rttm:
  diarization.write_rttm(rttm)

/Users/yoohoyeong/Desktop/AI_Practic/venv/lib/python3.12/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/Users/yoohoyeong/Desktop/AI_Practic/venv/lib/python3.12/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)
/Users/yoohoyeong/Desktop/

In [8]:
import pandas as pd
rttm_path = "../audio/싼기타_비싼기타.rttm"

df_rttm = pd.read_csv(
  rttm_path,
  sep=' ',
  header=None,
  names=['type', 'file', 'chnl', 'start', 'duration', 'C1', 'C2', 'speaker_id', 'C3', 'C4']
)

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN


In [ ]:
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']

display(df_rttm)